# LTSF Linear 

Implementation of the Linear Long Time Series Forcasting

## Imports

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim

# Settings seed for reproducibility 
np.random.seed(42)

dataset_path = '/kaggle/input/sl-project/dataset/'

In [ ]:
# Parameters of the model
lookback_window = 96
prediction_lengths = [96, 192, 336, 720]

# All the datasets
datasets = ['weather', 'exchange_rate', 'traffic', 'electricity', 'ETTh1', 'ETTm1']

# The LTSF-Linear model

In [30]:
# Defining the model as a class that inherits from nn.Module
# Taken from https://github.com/cure-lab/LTSF-Linear/

class LTSFLinear(nn.Module):

    def __init__(self, loopback_window, prediction_length):
        super(LTSFLinear, self).__init__()
        self.loopback_window = loopback_window
        self.prediction_length = prediction_length

        # The core of the model, a simple linear layer
        self.Linear = nn.Linear(self.loopback_window, self.prediction_length)

    def forward(self, x):
        x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        return x

In [31]:
# Function to convert the original dataframe to numpy arrays for pythorch model training

# Basically each element of X contains the sequence of data points of length lookback_window
# and the corresponding elements of y contains the sequence of data points of length prediction_length, 
# i.e. the data points to be predicted


def build_designMatrixAndPrediction(data, lookback_window, prediction_length):

    # Each row of X contains the sequence of data points of length lookback_window
    # Each row of y contains the sequence of data points of length prediction_length
    X = []
    y = []

    for i in range(len(data)-lookback_window-prediction_length+1):
        X.append(data[i : i+lookback_window])
        y.append(data[i+lookback_window : i+lookback_window+prediction_length])

    return np.array(X), np.array(y)

In [48]:
# Function to train the model with MSE Loss and Adam optimizer
# This is a modified version of the function provided in the github repository

def train_model(model, X_train, y_train, epochs, batch_size):

    # MSE Loss
    mse = nn.MSELoss()

    # Optimizer
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            outputs = model(batch_X)

            # Evaluate MSE loss and backpropagate
            loss = mse(outputs, batch_y)            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Log the loss every 10 epochs
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}] - Loss: {loss.item():.4f}')

In [ ]:
# Funciton to test the model
#  It outputts the MAE and RMSE of the model on the test data given as input

def test_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)

    #  Reshaping the data to the original shape
    # before evaluating the metrics
    y_test_inv = y_test.reshape(-1, y_test.shape[-1])
    y_pred_inv = y_pred.reshape(-1, y_pred.shape[-1])
    
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    
    return mae, rmse, y_test_inv, y_pred_inv

# Univariate TSs

We first train the model only on the dataset that have univariate Time Series.

In [47]:
# Univariate TS datasets

datasets_uni = ['exchange_rate', 'traffic', 'electricity']

In [ ]:
# For each dataset, train the model for each prediction length
# and store the results in a dictionary that later will be converted into a datatframe 

results = {}

for dataset in datasets_uni:

    print("\nDataset: ", dataset)

    df = pd.read_csv(dataset_path + dataset + '.csv', index_col='date', parse_dates=True)

    # Scaling all the columns in the range[0, 1]
    scaler = MinMaxScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
    
    results[dataset] = {}

    for pred_len in prediction_lengths:

        print(70 * "-")
        print(f"\nTraining LTSF-Linear for prediction length: {pred_len}")
        
        # Converting the dataframe to numpy arrays
        # such that the moel can be trained on it
        X, y = build_designMatrixAndPrediction(df, lookback_window, pred_len)
        
        # Splitting the data in 80% train and 20% test
        train_size = int(len(X) * 0.8)
        X_train, X_test = torch.FloatTensor(X[:train_size]), torch.FloatTensor(X[train_size:])
        y_train, y_test = torch.FloatTensor(y[:train_size]), torch.FloatTensor(y[train_size:])
        
        # Initializing and training the model
        modelLSTF = LTSFLinear(lookback_window, prediction_length = pred_len)
        
        train_model(modelLSTF, X_train, y_train, epochs = 100, batch_size = 32)
        
        # Testing the model and saving its metrics
        mae, rmse, y_test_inv, y_pred_inv = test_model(modelLSTF, X_test, y_test, scaler)
        
        results[dataset][pred_len] = {
            'MAE': mae,
            'RMSE': rmse
        }

In [45]:
# Creating a DataFrame from the results dictionary
multi_index = pd.MultiIndex.from_tuples(
    [(dataset, pred_len) for dataset in results for pred_len in results[dataset]],
    names=['Dataset', 'Prediction Length']
)

df_results = pd.DataFrame(
    [(results[dataset][pred_len]['MAE'], results[dataset][pred_len]['RMSE'])
     for dataset in results for pred_len in results[dataset]],
    index=multi_index,
    columns=['MAE', 'RMSE']
)

df_results

MAE      RMSE
Dataset       Prediction Length                    
exchange_rate 96                 0.044368  0.059844
              192                0.054688  0.073013
              336                0.079542  0.105306
              720                0.185081  0.224984
traffic       96                 0.040704  0.078620
              192                0.037695  0.075344
              336                0.039054  0.075822
electricity   96                 0.054440  0.080014
              192                0.058545  0.082661
              336                0.056093  0.081940
              720                0.058820  0.084733

In [6]:
# Writing df_results to a csv file
df_results.to_csv('results_uni.csv')

# Multivariate TSs

We first train the model only on the dataset that have multivariate Time Series.

Each file is a single Time Serie that have multiple feature to  be predicted.

In the same way that they do in the article, we will predict each feature separately using the LTSF-Linear model.

Since the datasets are too long, in order to not let the computation time exploding for high prediction windows we take only the last 10000 datapoints.

In [8]:
# Multivariate TS datasets

datasets_multi = ['weather', 'ETTh1', 'ETTm1']

In [ ]:
# For each dataset, train the model for each prediction length
# and store the results in a dictionary that later will be converted into a datatframe 

results_multi = {}

for dataset in datasets_multi:

    print("\nDataset: ", dataset)

    df = pd.read_csv(dataset_path + dataset + '.csv', index_col='date', parse_dates=True)
    
    # Taking the last 10000 datapoints of the Time Serie
    df = df.iloc[-10000:,:]

    # Scaling all the columns in the range[0, 1]
    scaler = MinMaxScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
    
    results_multi[dataset] = {}

    for pred_len in prediction_lengths:

        print(70 * "-")
        print(f"\nTraining LTSF-Linear for prediction length: {pred_len}")


        # For each prediction length, we train the model for each column of the dataset
        # And then we take the average of the MAE and RMSE of all the columns
        maes = []
        rmses = []

        for col in df.columns:
            print(70 * "-")
            print(f"\nTraining for column: {col}")            
            
            # Converting the dataframe to numpy arrays
            # such that the moel can be trained on it
            X, y = build_designMatrixAndPrediction(pd.DataFrame(df[col]), lookback_window, pred_len)
            
            # Split the data in 80% train and 20% test
            train_size = int(len(X) * 0.8)
            X_train, X_test = torch.FloatTensor(X[:train_size]), torch.FloatTensor(X[train_size:])
            y_train, y_test = torch.FloatTensor(y[:train_size]), torch.FloatTensor(y[train_size:])
            
            # Initializing and training the model
            modelLSTF = LTSFLinear(lookback_window, prediction_length = pred_len)
            
            train_model(modelLSTF, X_train, y_train, epochs = 50, batch_size = 32)
            
            # Testing the model and saving its metrics
            mae, rmse, y_test_inv, y_pred_inv = test_model(modelLSTF, X_test, y_test, scaler)
            maes.append(mae)
            rmses.append(rmse)
        
        # Saving the average of the MAE and RMSE of all the columns
        # In the dictionary
        results_multi[dataset][pred_len] = {
            'MAE': np.mean(maes),
            'RMSE': np.mean(rmses)
        }


In [14]:
# Creating a DataFrame from the results_multi dictionary
multi_index = pd.MultiIndex.from_tuples(
    [(dataset, pred_len) for dataset in results_multi for pred_len in results_multi[dataset]],
    names=['Dataset', 'Prediction Length']
)

df_results_multi = pd.DataFrame(
    [(results_multi[dataset][pred_len]['MAE'], results_multi[dataset][pred_len]['RMSE'])
     for dataset in results_multi for pred_len in results_multi[dataset]],
    index=multi_index,
    columns=['MAE', 'RMSE']
)

df_results_multi

MAE      RMSE
Dataset Prediction Length                    
weather 96                 0.089227  0.120001
        192                0.108004  0.144456
        336                0.143208  0.183607
        720                0.200576  0.245031
ETTh1   96                 0.076808  0.104071
        192                0.096826  0.125046
        336                0.109616  0.139671
        720                0.120480  0.155150
ETTm1   96                 0.073092  0.096906
        192                0.083562  0.107376
        336                0.087728  0.111352
        720                0.095922  0.121720

In [1]:
# Writing df_results_multi to a csv file
df_results_multi.to_csv('results_multi.csv')

# Finally merge the two dataframes with the metrics

In [17]:
df_results_final = pd.concat([df_results, df_results_multi])
df_results_final

MAE      RMSE
Dataset       Prediction Length                    
electricity   96                 0.054440  0.080014
              192                0.058545  0.082661
              336                0.056093  0.081940
              720                0.058820  0.084733
exchange_rate 96                 0.044368  0.059844
              192                0.054688  0.073013
              336                0.079542  0.105306
              720                0.185081  0.224984
traffic       96                 0.040704  0.078620
              192                0.037695  0.075344
              336                0.039054  0.075822
              720                0.053482  0.090758
weather       96                 0.089227  0.120001
              192                0.108004  0.144456
              336                0.143208  0.183607
              720                0.200576  0.245031
ETTh1         96                 0.076808  0.104071
              192                0.096826  0.125046
              336                0.109616  0.139671
              720                0.120480  0.155150
ETTm1         96                 0.073092  0.096906
              192                0.083562  0.107376
              336                0.087728  0.111352
              720                0.095922  0.121720

In [16]:
# Writing the results to file
df_results_final.to_csv('results_LTSF_Linear.csv')